# 3 — Workspace & Package Setup

> How to create a ROS2 workspace, understand the folder architecture, and create your own packages.

**Goal:** Build a proper ROS2 workspace from scratch and understand *why* every folder and file exists.

In [ ]:
#| default_exp workspace

## 3.1 What Is a Workspace?

A **workspace** is a directory where you develop, build, and run your ROS2 packages. Think of it as your project folder.

ROS2 uses an **overlay** system:

```
Layer 3: YOUR WORKSPACE  (~/ros2_ws)         ← what you create
   ↑ overlays
Layer 2: Extra packages   (/opt/ros/humble/)  ← apt-installed packages
   ↑ overlays  
Layer 1: Core ROS2        (/opt/ros/humble/)  ← base installation
```

When you `source` workspaces in order, later layers **override** earlier ones. This lets you develop modified versions of packages without touching the system install.

## 3.2 Create the Workspace

```bash
# Create the workspace directory
mkdir -p ~/ros2_ws/src
cd ~/ros2_ws
```

That's it! At this point your workspace is:

```
~/ros2_ws/
└── src/           ← your packages will live here
```

The `src/` folder is **critical** — `colcon` looks for packages inside `src/`.

## 3.3 Create Your First Package

ROS2 has a built-in tool to generate packages:

```bash
cd ~/ros2_ws/src

# Create a Python package
ros2 pkg create --build-type ament_python \
  --node-name my_first_node \
  --license Apache-2.0 \
  my_robot_pkg
```

### What `ros2 pkg create` generates:

```
~/ros2_ws/src/my_robot_pkg/
├── my_robot_pkg/              ← Python package (your code goes here)
│   ├── __init__.py
│   └── my_first_node.py      ← auto-generated starter node
├── test/                      ← test files
│   ├── test_copyright.py
│   ├── test_flake8.py
│   └── test_pep257.py
├── resource/                  ← ament resource index marker
│   └── my_robot_pkg
├── package.xml                ← package metadata & dependencies
├── setup.py                   ← Python build configuration
├── setup.cfg                  ← entry point configuration
└── LICENSE
```

## 3.4 Anatomy of Each File — Why It Exists

### `package.xml` — The Identity Card

This is the **most important file** in a ROS2 package. It declares:

```xml
<?xml version="1.0"?>
<package format="3">
  <name>my_robot_pkg</name>
  <version>0.1.0</version>
  <description>My first ROS2 robot package</description>
  <maintainer email="you@email.com">Your Name</maintainer>
  <license>Apache-2.0</license>

  <!-- Build tool -->
  <buildtool_depend>ament_python</buildtool_depend>

  <!-- Runtime dependencies -->
  <exec_depend>rclpy</exec_depend>
  <exec_depend>sensor_msgs</exec_depend>
  <exec_depend>cv_bridge</exec_depend>
  <exec_depend>opencv2</exec_depend>

  <!-- Test dependencies -->
  <test_depend>ament_copyright</test_depend>
  <test_depend>ament_flake8</test_depend>
  <test_depend>ament_pep257</test_depend>
  <test_depend>python3-pytest</test_depend>

  <export>
    <build_type>ament_python</build_type>
  </export>
</package>
```

**Why it matters:**
- `rosdep install` reads this to install missing OS packages
- `colcon` reads this to determine build order
- Other developers know what your package needs

### `setup.py` — Python Build Config

```python
from setuptools import find_packages, setup

package_name = 'my_robot_pkg'

setup(
    name=package_name,
    version='0.1.0',
    packages=find_packages(exclude=['test']),
    data_files=[
        # Install launch files
        ('share/' + package_name + '/launch',
         ['launch/camera_launch.py']),
        # Install config files
        ('share/' + package_name + '/config',
         ['config/camera_params.yaml']),
        # Required by ament
        ('share/ament_index/resource_index/packages',
         ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
    ],
    install_requires=['setuptools'],
    entry_points={
        'console_scripts': [
            'my_first_node = my_robot_pkg.my_first_node:main',
            # Add more nodes here as you create them
        ],
    },
)
```

**Key section:** `entry_points` maps command names to Python functions. When you run `ros2 run my_robot_pkg my_first_node`, ROS2 calls the `main()` function in `my_first_node.py`.

## 3.5 The Complete Workspace Architecture

Here's the full folder tree for a real project with multiple packages:

```
~/ros2_ws/                            ← WORKSPACE ROOT
├── src/                              ← SOURCE SPACE (your code)
│   ├── my_robot_pkg/                 ← Package 1: core robot nodes
│   │   ├── my_robot_pkg/             ← Python module
│   │   │   ├── __init__.py
│   │   │   ├── camera_node.py        ← camera subscriber/processor
│   │   │   ├── detector_node.py      ← deep learning detector
│   │   │   └── utils.py              ← shared utilities
│   │   ├── launch/                   ← launch files
│   │   │   ├── camera_launch.py
│   │   │   └── full_pipeline_launch.py
│   │   ├── config/                   ← configuration files
│   │   │   ├── camera_params.yaml
│   │   │   └── detector_params.yaml
│   │   ├── test/                     ← tests
│   │   ├── resource/
│   │   ├── package.xml
│   │   ├── setup.py
│   │   └── setup.cfg
│   │
│   ├── my_robot_msgs/                ← Package 2: custom message definitions
│   │   ├── msg/
│   │   │   └── Detection.msg
│   │   ├── srv/
│   │   │   └── TakeSnapshot.srv
│   │   ├── CMakeLists.txt            ← messages need CMake even for Python
│   │   └── package.xml
│   │
│   └── my_robot_description/         ← Package 3: robot URDF & meshes
│       ├── urdf/
│       ├── meshes/
│       ├── launch/
│       └── package.xml
│
├── build/                            ← BUILD SPACE (auto-generated)
├── install/                          ← INSTALL SPACE (auto-generated)
│   └── setup.bash                    ← source this!
└── log/                              ← LOG SPACE (auto-generated)
```

### Why This Structure?

| Folder | Purpose | Who creates it |
|--------|---------|----------------|
| `src/` | Your source packages | **You** |
| `build/` | Intermediate build files | **colcon** (auto) |
| `install/` | Built packages ready to use | **colcon** (auto) |
| `log/` | Build and test logs | **colcon** (auto) |

> **You only edit files in `src/`.** The other folders are regenerated on every `colcon build`.

## 3.6 Why Separate Packages?

Notice we have 3 packages in `src/`. Why not put everything in one?

| Package | Responsibility | Why separate |
|---------|---------------|-------------|
| `my_robot_pkg` | Python nodes (camera, detector, planner) | Main code, changes often |
| `my_robot_msgs` | Custom .msg and .srv definitions | Message definitions must be built *before* code that uses them |
| `my_robot_description` | URDF, meshes, robot model | Shared by simulation AND real robot |

**Separation of concerns** is a core ROS2 design principle. Packages should be:

1. **Single-purpose** — each package does one thing
2. **Reusable** — `my_robot_msgs` can be shared with other robots
3. **Independently buildable** — change one package without rebuilding everything

## 3.7 Build the Workspace

```bash
cd ~/ros2_ws

# Install any missing dependencies
rosdep install --from-paths src --ignore-src -r -y

# Build all packages
colcon build --symlink-install
```

### Understanding colcon build flags:

| Flag | What it does | Why use it |
|------|-------------|------------|
| `--symlink-install` | Creates symlinks instead of copies | Edit Python files without rebuilding! |
| `--packages-select pkg_name` | Build only one package | Faster iteration |
| `--packages-up-to pkg_name` | Build a package and its dependencies | When you need one package + deps |
| `--cmake-args -DCMAKE_BUILD_TYPE=Release` | Release mode for C++ | Faster execution |
| `--parallel-workers N` | Limit parallel jobs | Prevent OOM on limited hardware |

### After Building — Source the Workspace!

```bash
source install/setup.bash
```

If you forget this step, ROS2 won't find your packages. **This is the #1 source of "package not found" errors.**

## 3.8 Hands-On: Create Our Course Packages

Let's create the actual packages we'll use throughout the rest of this course:

```bash
cd ~/ros2_ws/src

# Package 1: Camera and vision nodes
ros2 pkg create --build-type ament_python \
  --license Apache-2.0 \
  --dependencies rclpy sensor_msgs cv_bridge \
  robot_vision

# Create standard subdirectories
mkdir -p robot_vision/launch
mkdir -p robot_vision/config
```

Now add our nodes to `setup.py`'s `entry_points` as we write them in subsequent notebooks.

## 3.9 Configuration Files (YAML)

Parameters can be defined in YAML files and loaded at launch time:

```yaml
# ~/ros2_ws/src/robot_vision/config/camera_params.yaml
camera_node:
  ros__parameters:
    video_device: "/dev/video0"
    frame_rate: 30.0
    image_width: 640
    image_height: 480
    pixel_format: "yuyv"

detector_node:
  ros__parameters:
    model_path: "yolov8n.pt"
    confidence_threshold: 0.5
    device: "cpu"   # or "cuda:0"
```

The `ros__parameters:` key (note the double underscore) tells ROS2 these are node parameters.

## 3.10 The .gitignore for a ROS2 Workspace

Never commit build artifacts:

```gitignore
# ROS2 build artifacts
build/
install/
log/

# Python
__pycache__/
*.pyc
*.egg-info/

# IDE
.vscode/
.idea/

# Deep learning models (large files)
*.pt
*.onnx
*.engine
```

## 3.11 Summary & Cheat Sheet

### Commands You'll Use Constantly

```bash
# Create workspace
mkdir -p ~/ros2_ws/src

# Create a package
cd ~/ros2_ws/src
ros2 pkg create --build-type ament_python --dependencies rclpy <pkg_name>

# Install dependencies
cd ~/ros2_ws
rosdep install --from-paths src --ignore-src -r -y

# Build
colcon build --symlink-install

# Source
source install/setup.bash

# Run a node
ros2 run <package_name> <executable_name>
```

### Key Takeaways

1. **Only edit in `src/`** — everything else is auto-generated
2. **Always source after building** — `source install/setup.bash`
3. **`--symlink-install`** — saves rebuilds for Python changes
4. **`package.xml`** — declares all dependencies (critical for `rosdep`)
5. **`setup.py` entry_points** — registers your Python scripts as ROS2 executables

---

**Next →** [Notebook 04: Your First ROS2 Node](04_first_node.ipynb)